In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "./data/multi_asset_etf_data.xlsx"

In [3]:
excess_returns_df = pd.read_excel(file_path, 
                                  sheet_name="excess returns", 
                                  parse_dates=["Date"],
                                  index_col="Date") # monthly excess returns

In [4]:
excess_returns_df

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.007027,0.042120,-0.000027,0.035908,0.014763,-0.001674,0.045614,0.040556,0.002993,0.035147,0.007618
2011-03-31,0.008755,0.026909,0.063224,-0.023555,0.000752,-0.001218,-0.010607,0.016995,0.005849,0.000448,0.012231
2011-04-30,0.048760,0.045514,0.027283,0.056214,0.015932,0.018369,0.046589,0.058627,0.018989,0.028917,0.023735
2011-05-31,-0.012945,-0.051124,-0.028800,-0.021461,0.001933,0.025654,0.010733,-0.040965,0.000600,-0.010615,0.003259
2011-06-30,0.000230,-0.042318,-0.009027,-0.011781,-0.005378,-0.004716,-0.030733,-0.042254,-0.010449,-0.016542,0.007876
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,-0.002198,0.024463,0.018388,0.044877,0.010472,0.003033,0.015780,0.064862,0.013104,0.023724,0.010570
2025-02-28,0.011376,-0.001250,0.008855,0.026915,0.007087,0.025382,0.035246,-0.042722,-0.008567,-0.015310,0.018957
2025-03-31,0.007510,0.019885,0.008497,-0.001004,-0.013701,0.000577,-0.026225,-0.063669,-0.016079,-0.058562,0.003954


# 1. Summary Statistics

In [ ]:
mean_m = excess_returns_df.mean()
mean_m_adj = mean_m + 0.01/12 #adjust data for risk-free rate
vol_m = excess_returns_df.std(ddof=1)

In [25]:
summary = pd.DataFrame({
    "Annualized Mean": mean_m_adj * 12,
    "Annualized Vol": vol_m * np.sqrt(12),
    "Annualized Sharpe": (mean_m_adj / vol_m) * np.sqrt(12),
    "Monthly Mean": mean_m_adj,
    "Monthly Vol": vol_m,
    "Monthly Sharpe": (mean_m_adj / vol_m)
})

In [7]:
summary = summary.sort_values("Annualized Sharpe", ascending=False)

In [8]:
summary

,Annualized Mean,Annualized Vol,Annualized Sharpe,Monthly Mean,Monthly Vol,Monthly Sharpe
SPY,0.128141,0.142839,0.897103,0.010678,0.041234,0.258971
HYG,0.041371,0.075928,0.544873,0.003448,0.021918,0.157291
IYR,0.074916,0.168675,0.444143,0.006243,0.048692,0.128213
PSP,0.092561,0.213370,0.433804,0.007713,0.061594,0.125228
EFA,0.061775,0.150903,0.409372,0.005148,0.043562,0.118176
TIP,0.020502,0.051115,0.401091,0.001708,0.014756,0.115785
QAI,0.019327,0.049073,0.393838,0.001611,0.014166,0.113691
IEF,0.016404,0.063442,0.258569,0.001367,0.018314,0.074642
EEM,0.029339,0.176164,0.166542,0.002445,0.050854,0.048077
DBC,-0.005292,0.166553,-0.031774,-0.000441,0.048080,-0.009172


# 2. Descriptive Analysis

In [9]:
corr = excess_returns_df.corr()

In [10]:
corr

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.191116,0.621673,0.602820,0.602555,0.580891,0.552557,0.526692,0.630276,0.439994,0.675151
DBC,0.191116,1.000000,0.511667,0.500922,0.461887,-0.300207,0.280518,0.453303,0.475311,0.432162,0.109006
EEM,0.621673,0.511667,1.000000,0.819925,0.691167,0.026704,0.584063,0.750109,0.774697,0.687751,0.378792
EFA,0.602820,0.500922,0.819925,1.000000,0.787191,0.042639,0.699292,0.895320,0.847864,0.845863,0.394821
HYG,0.602555,0.461887,0.691167,0.787191,1.000000,0.187258,0.739356,0.812157,0.807893,0.793518,0.538648
IEF,0.580891,-0.300207,0.026704,0.042639,0.187258,1.000000,0.316532,0.022436,0.179761,0.000815,0.754102
IYR,0.552557,0.280518,0.584063,0.699292,0.739356,0.316532,1.000000,0.749836,0.718529,0.754711,0.598742
PSP,0.526692,0.453303,0.750109,0.895320,0.812157,0.022436,0.749836,1.000000,0.873395,0.891687,0.408005
QAI,0.630276,0.475311,0.774697,0.847864,0.807893,0.179761,0.718529,0.873395,1.000000,0.866845,0.516670
SPY,0.439994,0.432162,0.687751,0.845863,0.793518,0.000815,0.754711,0.891687,0.866845,1.000000,0.381625


In [11]:
corr_pairs = corr.stack()
corr_pairs = corr_pairs[corr_pairs != 1]
max_pair, max_val = corr_pairs.idxmax(), corr_pairs.max()
min_pair, min_val = corr_pairs.idxmin(), corr_pairs.min()

In [12]:
print("Highest correlation:", max_pair, max_val)
print("Lowest correlation:", min_pair, min_val)

Highest correlation: ('EFA', 'PSP') 0.8953201243752301
Lowest correlation: ('DBC', 'IEF') -0.30020714105644963


In [13]:
annualized_mean = excess_returns_df.mean() * 12

In [14]:
# IEF: US Treasury 7-10y ETF
# BWX: International Treasury ETF

compare_df = pd.DataFrame({
    "Annualized Mean": annualized_mean[["TIP", "IEF", "BWX"]]
})

In [15]:
compare_df

,Annualized Mean
TIP,0.020502
IEF,0.016404
BWX,-0.007716


# 3. The MV frontier

In [24]:
#Tangency Portfolio - w^tan needs to sum to 1

cov_matrix = excess_returns_df.cov() 
cov_vals = cov_matrix.values
mean_vals = mean_m_adj.values

w_unnormalized = np.linalg.inv(cov_vals) @ mean_vals
w_tan = w_unnormalized / np.sum(w_unnormalized)

w_tan_df = pd.DataFrame({
    "Tangency Portfolio Weights": w_tan
}, index=excess_returns_df.columns)
print(w_tan_df)



     Tangency Portfolio Weights
BWX                   -0.842538
DBC                   -0.024318
EEM                    0.050096
EFA                    0.079902
HYG                    0.432276
IEF                    1.067763
IYR                   -0.327045
PSP                   -0.326764
QAI                   -0.684849
SPY                    1.209828
TIP                    0.365648


In [ ]:
#Sharpe Ratio vs Tangency Weights
weights_vs_sharpe = pd.DataFrame({
    "Tangency Weights": w_tan,
    "Annualized Sharpe": summary["Annualized Sharpe"]
}, index=excess_returns_df.columns)

print(weights_vs_sharpe)

     Tangency Weights  Annualized Sharpe
BWX         -6.117430          -0.093202
DBC         -0.112341          -0.031774
EEM          0.853591           0.166542
EFA          0.385285           0.409372
HYG          2.634922           0.544873
IEF          9.010591           0.258569
IYR         -2.382146           0.444143
PSP         -1.716842           0.433804
QAI        -13.616007           0.393838
SPY         10.729782           0.897103
TIP          1.330595           0.401091


# 4. TIPS